In [1]:
import pandas as pd
import os
import requests
from api import CENSUS_API_KEY
import ssl
from util.convertfile import *

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# xlsx_path = "~/Documents/UH/hon4350/water_quality/data/TRACT_ZIP_122024.xlsx"
# csv_path = "~/Documents/UH/hon4350/water_quality/data/TRACT_ZIP_122024.csv"

# xslx_to_csv(xlsx_path, csv_path)

In [4]:
# crosswalk = pd.read_csv("~/Documents/UH/hon4350/water_quality/data/TRACT_ZIP_122024.csv", dtype=str)
crosswalk = pd.read_csv("~/Documents/UH/hon4350/water_quality/data/ZIP_TRACT_122024.csv", dtype=str)
# crosswalk

In [5]:
crosswalk["STATE"] = crosswalk["TRACT"].str[:2]  # first 2 digits of TRACT
crosswalk_tx = crosswalk[crosswalk["STATE"] == "48"].copy()

crosswalk_tx["GEOID"] = crosswalk_tx["TRACT"]

crosswalk_tx_primary = crosswalk_tx[crosswalk_tx["RES_RATIO"].astype(float) > 0.5]

# primary --> this is for exploratory, actual dataset used is down below
crosswalk_tx_primary


,ZIP,TRACT,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO,STATE,GEOID
139082,73960,48421950200,TEXHOMA,TX,1.0,1.0,0.0,1.0,48,48421950200
140673,75009,48085030306,CELINA,TX,0.5020567772078615,0.6655896607431341,0.6343283582089553,0.5078751711993739,48,48085030306
140696,75011,48113013720,CARROLLTON,TX,0.6224520839671432,0.40350877192982454,0.5,0.5796038151137197,48,48113013720
140708,75014,48113014209,IRVING,TX,1.0,1.0,1.0,1.0,48,48113014209
140709,75015,48113014405,IRVING,TX,1.0,1.0,1.0,1.0,48,48113014405
...,...,...,...,...,...,...,...,...,...,...
154421,79954,48141001700,EL PASO,TX,1.0,1.0,1.0,1.0,48,48141001700
154422,79955,48141001700,EL PASO,TX,1.0,1.0,1.0,1.0,48,48141001700
154424,79995,48141003000,EL PASO,TX,1.0,1.0,1.0,1.0,48,48141003000
154425,79996,48141010326,EL PASO,TX,1.0,1.0,1.0,1.0,48,48141010326


In [6]:
years = list(range(2016, 2024))  # 2016–2023
STATE_FIPS = "48"  # Texas

# Store data per year
all_data = []

# ACS variables to fetch
acs_variables = ",".join([
    "NAME", "B01003_001E", "B19013_001E", "B19083_001E",
    "B03002_003E", "B03002_004E", "B03002_006E", "B03002_012E",
    "B25034_001E", "B25034_007E", "B25034_008E", "B25034_009E",
    "B25034_010E", "B25034_011E"
])

for year in years:
    print(f"Fetching data for {year}...")
    url = f"https://api.census.gov/data/{year}/acs/acs5"
    params = {
        "get": acs_variables,
        "for": "tract:*",
        "in": f"state:{STATE_FIPS}",
        "key": CENSUS_API_KEY
    }

    response = requests.get(url, params=params)
    if response.ok:
        data = response.json()
        df = pd.DataFrame(data[1:], columns=data[0])
        df["year"] = year
        all_data.append(df)
    else:
        print(f"Failed for {year}: {response.status_code}")

# Combine all years
acs_df = pd.concat(all_data, ignore_index=True)

# Rename columns
acs_df = acs_df.rename(columns={
    "NAME": "tract_name",
    "B01003_001E": "total_population",
    "B19013_001E": "median_household_income",
    "B19083_001E": "gini_index",
    "B03002_003E": "white_alone",
    "B03002_004E": "black_alone",
    "B03002_006E": "asian_alone",
    "B03002_012E": "hispanic_alone",
    "B25034_001E": "total_housing_units",
    "B25034_007E": "built_1970_1979",
    "B25034_008E": "built_1960_1969",
    "B25034_009E": "built_1950_1959",
    "B25034_010E": "built_1940_1949",
    "B25034_011E": "built_before_1940"
})

# Convert columns to numeric
cols_to_convert = [
    "total_population", "median_household_income", "gini_index",
    "white_alone", "black_alone", "asian_alone", "hispanic_alone",
    "total_housing_units", "built_1970_1979", "built_1960_1969",
    "built_1950_1959", "built_1940_1949", "built_before_1940"
]
acs_df[cols_to_convert] = acs_df[cols_to_convert].apply(pd.to_numeric, errors="coerce")

# Avoid division by zero
acs_df["total_population"] = acs_df["total_population"].replace(0, pd.NA)
acs_df["total_housing_units"] = acs_df["total_housing_units"].replace(0, pd.NA)

# Calculate race/ethnicity proportions
acs_df["pct_white"] = acs_df["white_alone"] / acs_df["total_population"]
acs_df["pct_black"] = acs_df["black_alone"] / acs_df["total_population"]
acs_df["pct_asian"] = acs_df["asian_alone"] / acs_df["total_population"]
acs_df["pct_hispanic"] = acs_df["hispanic_alone"] / acs_df["total_population"]

# Calculate % built before 1980
acs_df["housing_pre1980"] = (
    acs_df["built_1970_1979"] + acs_df["built_1960_1969"] +
    acs_df["built_1950_1959"] + acs_df["built_1940_1949"] +
    acs_df["built_before_1940"]
)
acs_df["pct_pre1980_housing"] = acs_df["housing_pre1980"] / acs_df["total_housing_units"]

# Round proportions
acs_df[["pct_white", "pct_black", "pct_asian", "pct_hispanic", "pct_pre1980_housing"]] = acs_df[
    ["pct_white", "pct_black", "pct_asian", "pct_hispanic", "pct_pre1980_housing"]
].round(3)


Fetching data for 2016...
Fetching data for 2017...
Fetching data for 2018...
Fetching data for 2019...
Fetching data for 2020...
Fetching data for 2021...
Fetching data for 2022...
Fetching data for 2023...


* weight each tract's data by its proportion in each ZIP.

In [7]:
# # Step 1: Standardize and construct GEOID
# acs_df["state"] = acs_df["state"].astype(str).str.zfill(2)
# acs_df["county"] = acs_df["county"].astype(str).str.zfill(3)
# acs_df["tract"] = acs_df["tract"].astype(str).str.zfill(6)
# acs_df["GEOID"] = acs_df["state"] + acs_df["county"] + acs_df["tract"]

# crosswalk["TRACT"] = crosswalk["TRACT"].str.zfill(11)
# crosswalk["ZIP"] = crosswalk["ZIP"].str.zfill(5)
# crosswalk["RES_RATIO"] = crosswalk["RES_RATIO"].astype(float)
# crosswalk["GEOID"] = crosswalk["TRACT"]

# # Step 2: Merge ACS with crosswalk using GEOID
# merged_df = pd.merge(crosswalk, acs_df, on="GEOID", how="inner")
# merged_df = merged_df[merged_df["RES_RATIO"] > 0.25]

# # Step 3: Convert relevant columns to numeric
# columns_to_weight = [
#     "total_population",
#     "median_household_income",
#     "white_alone",
#     "black_alone",
#     "asian_alone",
#     "hispanic_alone"
# ]

# for col in columns_to_weight:
#     merged_df[col] = pd.to_numeric(merged_df[col], errors="coerce")

# # Step 4: Apply RES_RATIO weights
# for col in columns_to_weight:
#     merged_df[f"{col}_weighted"] = merged_df[col] * merged_df["RES_RATIO"]

# # Step 5: Group by ZIP and aggregate weighted values
# zip_df = merged_df.groupby("ZIP", as_index=False).agg({
#     "total_population_weighted": "sum",
#     "median_household_income_weighted": "sum",
#     "white_alone_weighted": "sum",
#     "black_alone_weighted": "sum",
#     "asian_alone_weighted": "sum",
#     "hispanic_alone_weighted": "sum"
# })

# # Step 6: Recalculate demographic proportions and average income
# zip_df["pct_white"] = zip_df["white_alone_weighted"] / zip_df["total_population_weighted"]
# zip_df["pct_black"] = zip_df["black_alone_weighted"] / zip_df["total_population_weighted"]
# zip_df["pct_asian"] = zip_df["asian_alone_weighted"] / zip_df["total_population_weighted"]
# zip_df["pct_hispanic"] = zip_df["hispanic_alone_weighted"] / zip_df["total_population_weighted"]
# zip_df["median_household_income"] = zip_df["median_household_income_weighted"] / zip_df["total_population_weighted"]

# # Step 7: Final cleanup and rename
# zip_df = zip_df[[
#     "ZIP",
#     "total_population_weighted",
#     "median_household_income",
#     "pct_white",
#     "pct_black",
#     "pct_asian",
#     "pct_hispanic"
# ]].rename(columns={
#     "total_population_weighted": "total_population"
# })


* Each tract gets a single ZIP, the one where most residents live.

In [8]:
# Ensure strings and correct padding
acs_df["state"] = acs_df["state"].astype(str).str.zfill(2)
acs_df["county"] = acs_df["county"].astype(str).str.zfill(3)
acs_df["tract"] = acs_df["tract"].astype(str).str.zfill(6)

# Combine into GEOID
acs_df["GEOID"] = acs_df["state"] + acs_df["county"] + acs_df["tract"]

# Ensure Strings and correct padding for cross walk
crosswalk["ZIP"] = crosswalk["ZIP"].str.zfill(5)
crosswalk["TRACT"] = crosswalk["TRACT"].str.zfill(11)
crosswalk["RES_RATIO"] = crosswalk["RES_RATIO"].astype(float)

# Filter for Texas only (FIPS = '48') if needed
crosswalk["STATE"] = crosswalk["TRACT"].str[:2]
crosswalk_tx = crosswalk[crosswalk["STATE"] == "48"].copy()

# Keep only ZIP with highest RES_RATIO for each tract
crosswalk_best = crosswalk_tx.sort_values("RES_RATIO", ascending=False).drop_duplicates("TRACT")

# Add GEOID for merging
crosswalk_best["GEOID"] = crosswalk_best["TRACT"]

# Merge ACS data with ZIP data using GEOID
merged_df = pd.merge(acs_df, crosswalk_best[["GEOID", "ZIP"]], on="GEOID", how="left")


In [9]:
merged_df

,tract_name,total_population,median_household_income,gini_index,white_alone,black_alone,asian_alone,hispanic_alone,total_housing_units,built_1970_1979,built_1960_1969,built_1950_1959,built_1940_1949,built_before_1940,state,county,tract,year,pct_white,pct_black,pct_asian,pct_hispanic,housing_pre1980,pct_pre1980_housing,GEOID,ZIP
0,"Census Tract 3503, Harris County, Texas",6580,79363,0.3323,2808,624,615,2424,2117,1629,172,0,0,0,48,201,350300,2016,0.426748,0.094833,0.093465,0.368389,1801,0.850732,48201350300,77089
1,"Census Tract 4102, Harris County, Texas",5458,116506,0.4053,4057,225,391,742,3211,126,234,125,177,416,48,201,410200,2016,0.743313,0.041224,0.071638,0.135947,1078,0.335721,48201410200,NaN
2,"Census Tract 4113, Harris County, Texas",3396,79198,0.4928,2252,253,270,550,2667,390,374,0,0,44,48,201,411300,2016,0.663133,0.074499,0.079505,0.161955,808,0.302962,48201411300,NaN
3,"Census Tract 4119, Harris County, Texas",3458,111875,0.5391,2832,103,134,259,2282,122,184,55,211,830,48,201,411900,2016,0.818971,0.029786,0.038751,0.074899,1402,0.614373,48201411900,NaN
4,"Census Tract 4202, Harris County, Texas",2605,59955,0.3939,915,186,324,1091,1077,171,105,617,46,19,48,201,420200,2016,0.351248,0.071401,0.124376,0.41881,958,0.889508,48201420200,77025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48639,Census Tract 9504.02; Zapata County; Texas,2141,36250,0.5358,70,18,0,2053,1278,139,124,187,8,18,48,505,950402,2023,0.032695,0.008407,0.0,0.958898,476,0.372457,48505950402,78067
48640,Census Tract 9501; Zavala County; Texas,1033,26227,0.5078,10,0,0,1023,416,111,8,0,0,32,48,507,950100,2023,0.009681,0.0,0.0,0.990319,151,0.362981,48507950100,78829
48641,Census Tract 9502; Zavala County; Texas,1166,42270,0.3839,36,72,30,1028,501,0,51,37,0,20,48,507,950200,2023,0.030875,0.06175,0.025729,0.881647,108,0.215569,48507950200,78872
48642,Census Tract 9503.01; Zavala County; Texas,1917,52120,0.4366,219,0,0,1698,992,136,98,222,0,11,48,507,950301,2023,0.114241,0.0,0.0,0.885759,467,0.470766,48507950301,78839


In [10]:
merged_df[merged_df["total_population"].isna()].head()

,tract_name,total_population,median_household_income,gini_index,white_alone,black_alone,asian_alone,hispanic_alone,total_housing_units,built_1970_1979,built_1960_1969,built_1950_1959,built_1940_1949,built_before_1940,state,county,tract,year,pct_white,pct_black,pct_asian,pct_hispanic,housing_pre1980,pct_pre1980_housing,GEOID,ZIP
575,"Census Tract 9800, Midland County, Texas",<NA>,-666666666,-666666666.0,0,0,0,0,<NA>,0,0,0,0,0,48,329,980000,2016,<NA>,<NA>,<NA>,<NA>,0,<NA>,48329980000,79706
781,"Census Tract 9900, Jefferson County, Texas",<NA>,-666666666,-666666666.0,0,0,0,0,<NA>,0,0,0,0,0,48,245,990000,2016,<NA>,<NA>,<NA>,<NA>,0,<NA>,48245990000,NaN
804,"Census Tract 9900, Kenedy County, Texas",<NA>,-666666666,-666666666.0,0,0,0,0,<NA>,0,0,0,0,0,48,261,990000,2016,<NA>,<NA>,<NA>,<NA>,0,<NA>,48261990000,NaN
809,"Census Tract 9900, Kleberg County, Texas",<NA>,-666666666,-666666666.0,0,0,0,0,<NA>,0,0,0,0,0,48,273,990000,2016,<NA>,<NA>,<NA>,<NA>,0,<NA>,48273990000,NaN
1089,"Census Tract 9800, Potter County, Texas",<NA>,-666666666,-666666666.0,0,0,0,0,<NA>,0,0,0,0,0,48,375,980000,2016,<NA>,<NA>,<NA>,<NA>,0,<NA>,48375980000,79111


In [11]:
# Remove non-residential / special-use tracts
merged_df = merged_df[~merged_df["tract"].isin(["980000", "990000"])]

# Remove rows with invalid ACS placeholder codes
invalid_values = [-666666666, -666666666.0, -222222222, -222222222.0]

merged_df = merged_df[
    ~merged_df["median_household_income"].isin(invalid_values) &
    ~merged_df["gini_index"].isin(invalid_values)
]

In [12]:
merged_df = merged_df.rename(columns={"ZIP":"zip"})

In [13]:
# Ensure proper numeric types
merged_df["total_population"] = pd.to_numeric(merged_df["total_population"], errors="coerce")
merged_df["total_housing_units"] = pd.to_numeric(merged_df["total_housing_units"], errors="coerce")
merged_df["housing_pre1980"] = pd.to_numeric(merged_df["housing_pre1980"], errors="coerce")
merged_df["median_household_income"] = pd.to_numeric(merged_df["median_household_income"], errors="coerce")
merged_df["gini_index"] = pd.to_numeric(merged_df["gini_index"], errors="coerce")
merged_df["white_alone"] = pd.to_numeric(merged_df["white_alone"], errors="coerce")
merged_df["black_alone"] = pd.to_numeric(merged_df["black_alone"], errors="coerce")
merged_df["asian_alone"] = pd.to_numeric(merged_df["asian_alone"], errors="coerce")
merged_df["hispanic_alone"] = pd.to_numeric(merged_df["hispanic_alone"], errors="coerce")

# Compute weighted housing count (numerator)
# merged_df["pre1980_weighted"] = merged_df["housing_pre1980"]

# Group and aggregate by ZIP and year
zip_df = merged_df.groupby(["zip", "year"], as_index=False).agg({
    "total_population": "sum",
    "total_housing_units": "sum",
    "housing_pre1980": "sum",
    "median_household_income": "mean",
    "gini_index": "mean",
    "white_alone": "sum",
    "black_alone": "sum",
    "asian_alone": "sum",
    "hispanic_alone": "sum"
})

# Calculate race/ethnicity proportions
zip_df["pct_white"] = zip_df["white_alone"] / zip_df["total_population"]
zip_df["pct_black"] = zip_df["black_alone"] / zip_df["total_population"]
zip_df["pct_asian"] = zip_df["asian_alone"] / zip_df["total_population"]
zip_df["pct_hispanic"] = zip_df["hispanic_alone"] / zip_df["total_population"]

# Calculate % of homes built before 1980
zip_df["pct_pre1980_housing"] = zip_df["housing_pre1980"] / zip_df["total_housing_units"]

# Round for readability
zip_df[[
    "pct_white", "pct_black", "pct_asian", "pct_hispanic",
    "pct_pre1980_housing", "median_household_income", "gini_index"
]] = zip_df[[
    "pct_white", "pct_black", "pct_asian", "pct_hispanic",
    "pct_pre1980_housing", "median_household_income", "gini_index"
]].round(3)

zip_df


,zip,year,total_population,total_housing_units,housing_pre1980,median_household_income,gini_index,white_alone,black_alone,asian_alone,hispanic_alone,pct_white,pct_black,pct_asian,pct_hispanic,pct_pre1980_housing
0,73960,2016,3069,1335,994,52969.0,0.406,1739,14,0,1290,0.567,0.005,0.000,0.420,0.745
1,73960,2017,3067,1403,1058,54961.0,0.423,1712,8,0,1327,0.558,0.003,0.000,0.433,0.754
2,73960,2018,3058,1453,1144,56576.0,0.412,1721,9,0,1302,0.563,0.003,0.000,0.426,0.787
3,73960,2019,3059,1416,1122,51926.0,0.506,1635,35,0,1364,0.534,0.011,0.000,0.446,0.792
4,73960,2020,3046,1397,1071,57130.0,0.532,1623,4,35,1327,0.533,0.001,0.011,0.436,0.767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13464,79997,2019,5700,2428,2096,33500.0,0.450,453,83,56,5080,0.079,0.015,0.010,0.891,0.863
13465,79997,2020,6267,2597,2127,28789.0,0.516,405,89,47,5689,0.065,0.014,0.007,0.908,0.819
13466,79997,2021,5231,2604,2092,28532.0,0.532,495,97,46,4554,0.095,0.019,0.009,0.871,0.803
13467,79997,2022,6029,2603,1863,29946.0,0.562,492,60,67,5394,0.082,0.010,0.011,0.895,0.716


In [14]:
zip_df.describe()

,year,total_population,total_housing_units,housing_pre1980,median_household_income,gini_index,white_alone,black_alone,asian_alone,hispanic_alone,pct_white,pct_black,pct_asian,pct_hispanic,pct_pre1980_housing
count,13469.000000,13469.000000,13469.000000,13469.000000,13469.000000,13469.000000,13469.000000,13469.000000,13469.000000,13469.000000,13469.000000,13469.000000,13469.000000,13469.000000,13469.000000
mean,2019.722697,13285.200609,5254.465365,2142.863019,63685.794746,0.427325,5459.806741,1544.353330,602.568119,5326.141807,0.490164,0.100630,0.026995,0.357419,0.465599
std,2.280834,15159.317182,5582.966961,2500.042002,28272.297738,0.055316,6531.417127,3031.908027,1887.094488,8780.510068,0.263994,0.132592,0.050633,0.258572,0.232692
min,2016.000000,54.000000,47.000000,0.000000,9015.000000,0.144000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2018.000000,3455.000000,1562.000000,621.000000,44902.000000,0.391000,1432.000000,74.000000,3.000000,746.000000,0.267000,0.014000,0.001000,0.153000,0.291000
50%,2020.000000,6283.000000,2603.000000,1102.000000,57450.000000,0.424000,2978.000000,385.000000,56.000000,2013.000000,0.529000,0.051000,0.008000,0.280000,0.464000
75%,2022.000000,18223.000000,7298.000000,2585.000000,75250.000000,0.460000,6825.000000,1580.000000,370.000000,5637.000000,0.712000,0.134000,0.029000,0.514000,0.647000
max,2023.000000,125566.000000,40408.000000,18901.000000,250001.000000,0.706000,63289.000000,34863.000000,39818.000000,89893.000000,1.000000,0.947000,0.613000,1.000000,0.993000


In [15]:
zip_df["gini_index"].isnull().any()

False

In [16]:
zip_df.to_csv('~/Documents/UH/hon4350/water_quality/data/main/filtered_census_acs_5yr.csv', index=False)